In [68]:
import nltk
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
import os
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from collections import Counter

In [2]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [7]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

# X['x_train'] = train_pos + train_neg 
# test_x = test_pos + test_neg

<function str.index>

In [16]:
df = pd.DataFrame(np.array(train_pos + train_neg ), columns=['x_train'])

In [36]:
df['label'] = df.index.map(lambda idx: 1 if idx < 4000 else 0)


In [58]:
def process_tweet(tweet):
 
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [77]:
reviews= ""
i=0
for review in df['x_train']:
    words = process_tweet(review)
    for word in words:
        reviews = reviews + word +" "
reviews_l = reviews.split()
vocabulary = Counter(reviews_l)
unique_words = list(vocabulary.keys())

In [78]:
vocabulary['followfriday']
unique_words

['followfriday',
 'top',
 'engag',
 'member',
 'commun',
 'week',
 ':)',
 'hey',
 'jame',
 'odd',
 ':/',
 'pleas',
 'call',
 'contact',
 'centr',
 '02392441234',
 'abl',
 'assist',
 'mani',
 'thank',
 'listen',
 'last',
 'night',
 'bleed',
 'amaz',
 'track',
 'scotland',
 'congrat',
 'yeaaah',
 'yipppi',
 'accnt',
 'verifi',
 'rqst',
 'succeed',
 'got',
 'blue',
 'tick',
 'mark',
 'fb',
 'profil',
 '15',
 'day',
 'one',
 'irresist',
 'flipkartfashionfriday',
 'like',
 'keep',
 'love',
 'custom',
 'wait',
 'long',
 'hope',
 'enjoy',
 'happi',
 'friday',
 'lwwf',
 'second',
 'thought',
 '’',
 'enough',
 'time',
 'dd',
 'new',
 'short',
 'enter',
 'system',
 'sheep',
 'must',
 'buy',
 'jgh',
 'go',
 'bayan',
 ':d',
 'bye',
 'act',
 'mischiev',
 'etl',
 'layer',
 'in-hous',
 'wareh',
 'app',
 'katamari',
 'well',
 '…',
 'name',
 'impli',
 ':p',
 'influenc',
 'big',
 '...',
 'juici',
 'selfi',
 'follow',
 'perfect',
 'alreadi',
 'know',
 "what'",
 'great',
 'opportun',
 'junior',
 'triathle

In [79]:

def extract_features(tweet, freqs):
  
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    

    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        
        x[0,1] += freqs.get((word,1),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)

    assert(x.shape == (1, 3))
    return x

In [82]:
tmp1 = extract_features(df['x_train'], vocabulary)
print(tmp1)

TypeError: expected string or bytes-like object